In [ ]:
import json
import os
import pandas as pd 
from matplotlib import pyplot as plt
from filter_optimization.filter_optimization_task import extract_biomass_data

plt.rcParams['font.size'] = 18

In [ ]:
df = extract_biomass_data(56, '2020-06-01', '2020-08-31', 0.99)
df.hour = (df.hour + 2) % 24 # time zone hack
plt.figure(figsize=(10, 5))
mask = df.estimated_k_factor > 0.0
df[mask].groupby('hour')['estimated_weight_g'].agg(lambda x: x.mean()).plot()
plt.ylabel('Average Weight (g)')
plt.grid()

In [ ]:
df = extract_biomass_data(60, '2020-08-12', '2020-08-26', 0.99)
df.hour = (df.hour + 2) % 24
plt.figure(figsize=(10, 5))
mask = df.estimated_k_factor > 0.0
df[mask].groupby('hour')['estimated_weight_g'].agg(lambda x: x.mean()).plot()
plt.grid()

In [ ]:
df = extract_biomass_data(100, '2020-06-01', '2020-08-31', 0.99)
df.hour = (df.hour + 2) % 24
plt.figure(figsize=(10, 5))
mask = df.estimated_k_factor > 0.0
df[mask].groupby('hour')['estimated_weight_g'].agg(lambda x: x.mean()).plot()
plt.grid()

In [ ]:
df = extract_biomass_data(88, '2020-02-01', '2020-03-06', 0.99)
df.hour = (df.hour + 2) % 24
plt.figure(figsize=(10, 5))
mask = df.estimated_k_factor > 0.0
df[mask].groupby('hour')['estimated_weight_g'].agg(lambda x: x.mean()).plot()
plt.grid()

In [ ]:
df = extract_biomass_data(95, '2020-04-01', '2020-07-30', 0.99)
df.hour = (df.hour + 2) % 24
plt.figure(figsize=(10, 5))
mask = df.estimated_k_factor > 0.0
df[mask].groupby('hour')['estimated_weight_g'].agg(lambda x: x.mean()).plot()
plt.grid()

In [ ]:
df = extract_biomass_data(131, '2020-08-12', '2020-08-26', 0.99)
df.hour = (df.hour + 2) % 24
plt.figure(figsize=(10, 5))
mask = df.estimated_k_factor > 0.0
df[mask].groupby('hour')['estimated_weight_g'].agg(lambda x: x.mean()).plot()
plt.grid()

In [ ]:
df = extract_biomass_data(132, '2020-07-01', '2020-07-26', 0.99)
df.hour = (df.hour + 2) % 24
plt.figure(figsize=(10, 5))
mask = df.estimated_k_factor > 0.0
df[mask].groupby('hour')['estimated_weight_g'].agg(lambda x: x.mean()).plot()
plt.grid()

In [ ]:
df = extract_biomass_data(132, '2020-08-27', '2020-09-03', 0.99)

plt.figure(figsize=(20, 10))
mask = df.estimated_k_factor > 1.08
df[mask].groupby('hour')['estimated_weight_g'].agg(lambda x: x.mean()).plot()
# df.groupby('hour')['estimated_weight_g'].agg(lambda x: x.median()).plot()
plt.grid()

In [ ]:
df = extract_biomass_data(132, '2020-08-20', '2020-08-27', 0.99)

plt.figure(figsize=(20, 10))
mask = df.estimated_k_factor > 1.08
df[mask].groupby('hour')['estimated_weight_g'].agg(lambda x: x.mean()).plot()
plt.grid()

In [ ]:
depths = []
for idx, row in df.iterrows():
    ann, cm = row.annotation, row.camera_metadata
    wkps = pixel2world(ann['leftCrop'], ann['rightCrop'], cm)
    depth = np.median([wkp[1] for wkp in wkps.values()])
    depths.append(depth)
df['depth'] = depths

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(20, 10))

amts = [2000, 3000, 4000, 5000]

for amt in amts:
    df1 = df[(df.estimated_weight_g < amt) & (df.estimated_weight_g > amt - 1000) ].groupby('date')['depth'].agg(lambda x: x.mean() )
    df1.index = pd.to_datetime(df1.index)
    
    df_before = df1[(df1.index < '2020-08-01')]
    df_after = df1[(df1.index >= '2020-08-01')]
    x1 = np.arange(len(df_before)) # = array([0, 1, 2, ..., 3598, 3599, 3600])
    fit1 = np.polyfit(x1, df_before.values, 1)
    fit_fn1 = np.poly1d(fit1)
    
    x2 = np.arange(len(df_after)) # = array([0, 1, 2, ..., 3598, 3599, 3600])
    fit2 = np.polyfit(x2, df_after.values, 1)
    fit_fn2 = np.poly1d(fit2)

    axes.plot(df1.index, df1.values, label = amt)
    axes.plot(df_before.index, fit_fn1(x1), 'k--')
    axes.plot(df_after.index, fit_fn2(x2), 'k--')

axes.set_xlabel('Date')
axes.set_ylabel('Depth from Camera')
axes.set_title('Varholmen Pen 6: Average Depth from Camera By Weight')

axes.legend()
    
# df2 = df[(df.estimated_weight_g < pct75) & (df.estimated_weight_g > pct75 - 1000) ].groupby('date')['depth'].agg(lambda x: x.mean() )
# df2.index = pd.to_datetime(df2.index)

# df5 = df2 - df1
# df6 = df5.rolling(window=7).mean()

# df3 = df1.rolling(window=7).mean()
# df4 = df2.rolling(window=7).mean()

#plt.plot(df[df.estimated_k_factor > 1.08].groupby('date')['depth'].agg(lambda x: x.mean() ), color = 'red')
# plt.plot(df5, color = 'green')

# x2 = np.arange(len(df2)) # = array([0, 1, 2, ..., 3598, 3599, 3600])
# fit2 = np.polyfit(x2, df2.values, 1)
# fit_fn2 = np.poly1d(fit2)


# plt.plot(df2.index, df2.values, color = 'red')
# plt.plot(df2.index, fit_fn2(x2), 'k--')

In [ ]:
df_after.values

In [ ]:
df[mask].groupby('hour')['estimated_weight_g'].agg(lambda x: x.shape[0]).plot()

In [ ]:
from research.weight_estimation.keypoint_utils.optics import pixel2world
import numpy as np
from research.weight_estimation.keypoint_utils import body_parts

def get_raw_3d_coordinates(keypoints, cm):
    wkps = pixel2world([item for item in keypoints['leftCrop'] if item['keypointType'] != 'BODY'],
                       [item for item in keypoints['rightCrop'] if item['keypointType'] != 'BODY'],
                       cm)

    all_wkps = np.array([list(wkps[bp]) for bp in body_parts.core_body_parts])
    return all_wkps

In [ ]:
depths = []
for idx, row in df.iterrows():
    try:
        wkps = get_raw_3d_coordinates(row.annotation, row.camera_metadata)
        depth = np.median(wkps[:, 1])
    except:
        depth = None
    depths.append(depth)
    

In [ ]:
df['depth'] = depths

In [ ]:
plt.hist(df.depth, bins=20)
plt.show()

In [ ]:
plt.hist(df.depth, bins=20)
plt.show()

In [ ]:
plt.hist(df.depth, bins=20)
plt.show()

In [ ]:
df = extract_biomass_data(124, '2020-08-10', '2020-08-29', 0.99)

plt.figure(figsize=(20, 10))
mask = df.estimated_k_factor > 1.25
df[mask].groupby('hour')['estimated_weight_g'].agg(lambda x: x.mean()).plot()
plt.grid()

In [ ]:
df = extract_biomass_data(124, '2020-08-10', '2020-08-29', 0.99)

plt.figure(figsize=(15, 8))
mask = df.estimated_k_factor > 1.3
df[mask].groupby('hour')['estimated_weight_g'].agg(lambda x: x.mean()).plot()
plt.grid()

In [ ]:
plt.figure(figsize=(15, 8))
df[mask].groupby('hour')['estimated_weight_g'].agg(lambda x: x.shape[0]).plot()
plt.grid()

In [ ]:
rds